In [3]:
!pip install matplotlib tcia_utils monai rt_utils scipy

In [ ]:
import os
import numpy as np
import torch
import matplotlib.pyplot as plt
from tcia_utils import nbia
from monai.bundle import ConfigParser, download
from monai.transforms import LoadImage, LoadImaged, Orientation, Orientationd, EnsureChannelFirst, EnsureChannelFirstd, Compose
from rt_utils import RTStructBuilder
from scipy.ndimage import label, measurements
import json

In [ ]:
!mkdir data

In [ ]:
datadir = 'data'

In [ ]:
cart_name = "nbia-56561691129779503"
cart_data = nbia.getSharedCart(cart_name)
df = nbia.downloadSeries(cart_data, format="df", path = datadir)

In [ ]:
CT_folder = os.path.join(datadir, '1.3.6.1.4.1.14519.5.2.1.3320.3273.193828570195012288011029757668')

In [ ]:
image_loader = LoadImage(image_only=True)
CT = image_loader(CT_folder)

In [ ]:
CT.meta

In [ ]:
CT_coronal_slice = CT[:,256].cpu().numpy()

In [ ]:
plt.figure(figsize=(3,8))
plt.pcolormesh(CT_coronal_slice.T, cmap='Greys_r')
plt.colorbar(label='HU')
plt.axis('off')
plt.show()

In [ ]:
CT.shape

In [ ]:
channel_transform = EnsureChannelFirst()
CT = channel_transform(CT)
CT.shape

In [ ]:
orientation_transform = Orientation(axcodes=('LPS'))
CT = orientation_transform(CT)

In [ ]:
CT_coronal_slice = CT[0,:,256].cpu().numpy()

In [ ]:
plt.figure(figsize=(3,8))
plt.pcolormesh(CT_coronal_slice.T, cmap='Greys_r')
plt.colorbar(label='HU')
plt.axis('off')
plt.show()

In [ ]:
preprocessing_pipeline = Compose([
    LoadImage(image_only=True),
    EnsureChannelFirst(),
    Orientation(axcodes='LPS')
])

In [ ]:
CT = preprocessing_pipeline(CT_folder)
CT_coronal_slice = CT[0,:,256].cpu().numpy()

In [ ]:
plt.figure(figsize=(3,8))
plt.pcolormesh(CT_coronal_slice.T, cmap='Greys_r')
plt.colorbar(label='HU')
plt.axis('off')
plt.show()

In [ ]:
data = {'image': CT_folder, 'some_other_key': 42}

In [ ]:
preprocessing_pipeline = Compose([
    LoadImaged(keys='image', image_only=True),
    EnsureChannelFirstd(keys='image'),
    Orientationd(keys='image',axcodes='LPS')
])

In [ ]:
data = preprocessing_pipeline(data)

In [23]:
data

{'image': metatensor([[[[-1024., -1024., -1024.,  ..., -1024., -1024., -1024.],
           [-1024., -1024., -1024.,  ..., -1024., -1024., -1024.],
           [-1024., -1024., -1024.,  ..., -1024., -1024., -1024.],
           ...,
           [-1024., -1024., -1024.,  ..., -1024., -1024., -1024.],
           [-1024., -1024., -1024.,  ..., -1024., -1024., -1024.],
           [-1024., -1024., -1024.,  ..., -1024., -1024., -1024.]],
 
          [[-1024., -1024., -1024.,  ..., -1024., -1024., -1024.],
           [-1024., -1024., -1024.,  ..., -1024., -1024., -1024.],
           [-1024., -1024., -1024.,  ..., -1024., -1024., -1024.],
           ...,
           [-1024., -1024., -1024.,  ..., -1024., -1024., -1024.],
           [-1024., -1024., -1024.,  ..., -1024., -1024., -1024.],
           [-1024., -1024., -1024.,  ..., -1024., -1024., -1024.]],
 
          [[-1024., -1024., -1024.,  ..., -1024., -1024., -1024.],
           [-1024., -1024., -1024.,  ..., -1024., -1024., -1024.],
           

In [24]:
!wget --content-disposition https://api.ngc.nvidia.com/v2/models/nvidia/monaitoolkit/monai_wholebody_ct_segmentation/versions/0.1.9/zip -O monai_wholebody_ct_segmentation.zip

--2024-05-06 08:25:19--  https://api.ngc.nvidia.com/v2/models/nvidia/monaitoolkit/monai_wholebody_ct_segmentation/versions/0.1.9/zip
Resolving api.ngc.nvidia.com (api.ngc.nvidia.com)... 18.236.42.48, 44.228.149.205
Connecting to api.ngc.nvidia.com (api.ngc.nvidia.com)|18.236.42.48|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://files.ngc.nvidia.com/org/nvidia/team/monaitoolkit/models/monai_wholebody_ct_segmentation/versions/0.1.9/files.zip?versionId%3D2B7Nna7MLNUNWFP1XuzyT_Mz42a7pX0T&Expires=1715070320&Signature=QYiVl3vMSpxu4PmEjIo0DpUoeIP-ohgz4rYfLfAvmnwgTTrSjePwQN-HrhzR-jzK3yJtYIJqi5b8PpP0rema8NVRAwP~MxQUmJ5WuODp-ZqGZYesd4Fpsvh19q-mzPIW8Kj3hUgDKgfcroHJL95LvTWq6AXA9CeE5Vz~dK4yCA8J-wHtO1bBxmKLtp-GtdWRNR-T3R3m2NIscpuDCDhZAc4zj3eBN2uvgOUBoToi9cDmVgAqjlLpvFrxtzlTW8JDZlMh7h3NF6fzgZfcf4awO8U0vwRP0r8NYXHcQ--bzUdBrk9kBmwuCwFaD6UWSNGm9dg96DKTfQi6M8Gb47FU-g__&Key-Pair-Id=KCX06E8E9L60W [following]
--2024-05-06 08:25:20--  https://files.ngc.nvidia.com/org/nv

In [ ]:
!unzip monai_wholebody_ct_segmentation.zip -d data/m_wb_ct_sgm

Archive:  monai_wholebody_ct_segmentation.zip
replace data/m_wb_ct_sgm/LICENSE? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
model_path = os.path.join(datadir, 'm_wb_ct_sgm', 'models', 'model_lowres.pt')
config_path = os.path.join(datadir, 'm_wb_ct_sgm', 'configs', 'inference.json')

In [ ]:
config = ConfigParser()
config.read_config(config_path)

In [ ]:
preprocessing = config.get_parsed_content("preprocessing")

In [ ]:
data = preprocessing({'image': CT_folder})

In [ ]:
model = config.get_parsed_content("network")